# 处理样例

In [1]:
import csv
import re
import pandas as pd

In [ ]:
# 假设你的文本数据存储在一个字符串变量data中
file_name = 'output2'
with open(f'{file_name}.txt', 'r') as f:
    file = f.read()

# 使用正则表达式来分割请求和响应
# 使用正则表达式匹配以 GET 开始并以 HTTP/1.1 结尾的部分
matches = re.findall(r'((GET|POST)\s.*?HTTP/1\.1[\s\S]*?(?=(GET|POST)\s|$))', file)
pd.DataFrame(matches)

data =[]
for i in range(len(matches)):
    # 使用 split 方法分割字符串
    parts = matches[i][0].split('HTTP/1.1 200 OK')

    # parts 现在包含两部分，parts[0] 是请求部分，parts[1] 是响应部分
    req_part = parts[0].strip()
    req_part = re.split(r'\n\s*\n', req_part)
    req_header = req_part[0].strip()
    req_body = '' if len(req_part) == 1 else req_part[1].strip()
    
    if len(parts) > 1:
        rsp_part = ('HTTP/1.1 200 OK' + parts[1]).strip()
        rsp_part = re.split(r'\n\s*\n', rsp_part)
        rsp_header = rsp_part[0].strip()
        rsp_body = '' if len(rsp_part) == 1 else rsp_part[1].strip()
    else:
        rsp_header = ''
        rsp_body = ''

    data.append({"req header": req_header,
                 "req body":req_body, 
                 "rsp header": rsp_header,
                 "rsp body": rsp_body,
                 "Label": file_name})
df = pd.DataFrame(data)
df.to_excel(f'{file_name}.xlsx', index=False)
df

# 处理样本

In [192]:
import json

data = json.load(open("output (2).json"))
data[0]['_source']

{'layers': {'frame': {'frame.encap_type': '1',
   'frame.time': 'Aug 26, 2021 15:21:45.822235000 CST',
   'frame.offset_shift': '0.000000000',
   'frame.time_epoch': '1629962505.822235000',
   'frame.time_delta': '0.000000000',
   'frame.time_delta_displayed': '0.000000000',
   'frame.time_relative': '0.000000000',
   'frame.number': '1',
   'frame.len': '588',
   'frame.cap_len': '588',
   'frame.marked': '0',
   'frame.ignored': '0',
   'frame.protocols': 'eth:ethertype:ip:tcp:http'},
  'eth': {'eth.dst': '00:50:56:f1:c2:d8',
   'eth.dst_tree': {'eth.dst_resolved': 'VMware_f1:c2:d8',
    'eth.dst.oui': '20566',
    'eth.dst.oui_resolved': 'VMware, Inc.',
    'eth.addr': '00:50:56:f1:c2:d8',
    'eth.addr_resolved': 'VMware_f1:c2:d8',
    'eth.addr.oui': '20566',
    'eth.addr.oui_resolved': 'VMware, Inc.',
    'eth.dst.lg': '0',
    'eth.lg': '0',
    'eth.dst.ig': '0',
    'eth.ig': '0'},
   'eth.src': '00:0c:29:b6:00:b7',
   'eth.src_tree': {'eth.src_resolved': 'VMware_b6:00:b7',
 

In [212]:
tables =[]
for i in range(len(data)):
    if data[i]['_source']['layers'].get('http'):
        tmp = {'label':''}
        if 'http.request' in data[i]['_source']['layers']['http']:
            first_key = next(iter(data[i]['_source']['layers']['http']))
            tmp['method'] = data[i]['_source']['layers']['http'][first_key]['http.request.method']
            if tmp['method'] == 'POST':
                tmp['req-contlen'] = int(data[i]['_source']['layers']['http']['http.content_length_header'].strip())
            else:
                tmp['req-contlen'] = ''
        else:
            tmp['rsp-contlen'] = data[i]['_source']['layers']['http']['http.content_length_header'].strip()
            tmp['Date'] = data[i]['_source']['layers']['http']['http.date'].strip()
        tables.append(tmp)
df = pd.DataFrame(tables)
df

,label,method,req-contlen,rsp-contlen,Date
0,,GET,,NaN,NaN
1,,NaN,NaN,5543,"Thu, 26 Aug 2021 07:21:40 GMT"
2,,GET,,NaN,NaN
3,,NaN,NaN,5607,"Thu, 26 Aug 2021 07:21:52 GMT"
4,,POST,395,NaN,NaN


In [213]:
import numpy as np

merged_tables = []

i = 0
while i < len(df):
    if pd.notna(df.loc[i, 'method']) and i + 1 < len(df) and pd.isna(df.loc[i + 1, 'method']):
        # 合并第i行和第i+1行
        merged_row = df.loc[i].combine_first(df.loc[i + 1])
        merged_tables.append(merged_row)
        i += 2  # 跳过下一行
    else:
        # 如果不满足条件，直接添加当前行
        merged_tables.append(df.loc[i])
        i += 1

# 创建新的数据框
merged_df = pd.DataFrame(merged_tables).reset_index(drop=True)
merged_df

,label,method,req-contlen,rsp-contlen,Date
0,,GET,,5543,"Thu, 26 Aug 2021 07:21:40 GMT"
1,,GET,,5607,"Thu, 26 Aug 2021 07:21:52 GMT"
2,,POST,395,NaN,NaN


In [ ]:
#!/bin/bash

# 定义pcap文件名
PCAP_FILE="http_stage2.pcap"

# 定义流的范围
START_STREAM=0
END_STREAM=3511

# 循环处理每个流
for ((stream=$START_STREAM; stream<=$END_STREAM; stream++))
do
    # 定义输出文件名
    OUTPUT_FILE="output_${stream}.json"
    
    # 使用tshark命令提取特定流的数据并保存到文件
    tshark -r $PCAP_FILE -Y "tcp.stream eq $stream" -T json > $OUTPUT_FILE
    
    # 可选：打印当前处理的流号
    echo "Processed tcp.stream eq $stream"
done

echo "All streams processed."

## 一键处理

In [8]:
import numpy as np
import json
import pandas as pd

tables =[]
def getTable(file):
    data = json.load(open(file))
    data[0]['_source']
    tables=[]
    for i in range(len(data)):
        if data[i]['_source']['layers'].get('http'):
            tmp = {'label':''}
            httpLayer = data[i]['_source']['layers']['http']
            if 'http.request' in httpLayer:
                first_key = next(iter(httpLayer))
                tmp['method'] = httpLayer[first_key]['http.request.method']
                if httpLayer.get('http.content_length_header') and tmp['method'] == 'POST':
                    tmp['req-contlen'] = int(httpLayer['http.content_length_header'].strip())
                else:
                    tmp['req-contlen'] = ''
            else:
                if httpLayer.get('http.content_length_header'):
                    tmp['req-contlen'] = int(httpLayer['http.content_length_header'].strip())
                else:
                    tmp['req-contlen'] = ''
                if httpLayer.get('http.date'):
                    tmp['Date'] = data[i]['_source']['layers']['http']['http.date'].strip()
                    tmp['Time'] = data[i]['_source']['layers']['frame']['frame.time'].strip()
                else:
                    tmp['Date'] = ''
                    tmp['Time'] = data[i]['_source']['layers']['frame']['frame.time'].strip()
            tables.append(tmp)
    df = pd.DataFrame(tables)
    # print(df)
    merged_tables = []
    i = 0
    while i < len(df):
        if pd.notna(df.loc[i, 'method']) and i + 1 < len(df) and pd.isna(df.loc[i + 1, 'method']):
            # 合并第i行和第i+1行
            merged_row = df.loc[i].combine_first(df.loc[i + 1])
            merged_tables.append(merged_row)
            i += 2  # 跳过下一行
        else:
            # 如果不满足条件，直接添加当前行
            merged_tables.append(df.loc[i])
            i += 1

    # 创建新的数据框
    merged_df = pd.DataFrame(merged_tables).reset_index(drop=True)
     # 检查是否存在 'Data' 和 'Time' 列，如果不存在则添加并填充空值
    if 'rsp-contlen' not in merged_df.columns:
        merged_df['rsp-contlen'] = ''   
    if 'Date' not in merged_df.columns:
        merged_df['Date'] = ''
    if 'Time' not in merged_df.columns:
        merged_df['Time'] = ''

    return merged_df

df = getTable(f"Data/datacon2021_traffic_cs2_http/output_0.json")
df

,label,method,req-contlen,rsp-contlen,Date,Time
0,,GET,,4085,"Tue, 28 Sep 2021 07:23:49 GMT","Sep 28, 2021 15:23:51.555030000 CST"


In [9]:
for i in range(1,3511):
    print('*'*50+str(i)+"*"*50)
    tf = getTable(f"Data/datacon2021_traffic_cs2_http/tmp/output_{i}.json")
    print(tf)
    df = pd.concat([df,tf])

**************************************************1**************************************************
  label method req-contlen rsp-contlen Date Time
0          GET                                  
**************************************************2**************************************************
  label method req-contlen rsp-contlen Date Time
0          GET                                  
**************************************************3**************************************************
  label method req-contlen rsp-contlen Date  \
0          GET                       0        

                                  Time  
0  Sep 28, 2021 15:23:50.241988000 CST  
**************************************************4**************************************************
  label method req-contlen rsp-contlen                           Date  \
0          GET                    4073  Tue, 28 Sep 2021 07:23:50 GMT   

                                  Time  
0  Sep 28, 2021 15:23:50.701522

KeyError: 'http.content_length_header'

In [7]:
df

,label,method,req-contlen,rsp-contlen,Date,Time
0,,GET,,4085,"Tue, 28 Sep 2021 07:23:49 GMT","Sep 28, 2021 15:23:51.555030000 CST"
0,,GET,,,,
0,,GET,,,,
0,,GET,,0,,"Sep 28, 2021 15:23:50.241988000 CST"
0,,GET,,4073,"Tue, 28 Sep 2021 07:23:50 GMT","Sep 28, 2021 15:23:50.701522000 CST"
